In [177]:
n = 6
d = 3
P = Polyhedron([[random() for _ in range(d)] for i in range(n)])
P

A 3-dimensional polyhedron in RDF^3 defined as the convex hull of 6 vertices (use the .plot() method to plot)

In [178]:
G = P.graph()
G.is_inscribable()

True

In [211]:
import re
def scip_str(varis, eqns, obj, bounds, maximize=True):
    firstletters = ''.join(set([str(_)[0] for _ in varis]))
    clean=re.compile(r'([\ ,-][0-9]+\.*[0-9]*)\*(['+firstletters+'])')
    #p=re.compile(r'\(.*e...\)')

    if maximize:
        out='Maximize'
    else:
        out='Minimize'
    
    out+='\n obj: ' + str(obj) + '\nSubject to\n'
    for i, eq in enumerate(eqns):
        out+= clean.sub(r"\1\2", "eq"+str(i)+": "+str(eq)+"\n")
    out+="Bounds\n"
    for v in varis:
        out+='{} <= {} <={}\n'.format(bounds[v][0], v, bounds[v][1])
    return out
def solution(loes, varis):
    los=open(loes)
    solu=dict()
    p=dict()
    for v in varis:
        solu[v]=0
    with los as file:
        for line in file.readlines():
            if 'no solution available' in line:
                return 0
            begins=[str(v)[0] for v in varis]
            if line[0] in begins:
                solu[SR(line.split()[0])]=RR(line.split()[1])
    los.close()
    return solu


In [200]:
x = dict()
bounds = dict()
for i in range(P.n_vertices()):
    for j in range(d):
        x[i,j] = var('x_{}_{}'.format(i,j))
        bounds[x[i,j]] = [-1, 1]
m = var('m')
bounds[m] = [0.0000015,20]

In [201]:
eqns = []
print binomial(n, d+1)
for fac in P.faces(d-1):
    face =  [_.index() for _ in fac.vertices()]
    #print face
    for i in range(P.n_vertices()):
        if i not in face:
            c = [i]+face
            sig = Matrix([[1] +P.vertices_list()[i] for i in c]).det().sign()
            #print sig
            assert sig in [1, -1]
            eqns.append((sig*Matrix([[1] +[x[i,j] for j in range(d)] for i in c]).det()-m) >=0)
for j in range(d):
    eqns.append(x[0,j] == int(j==0))
eqns.append(x[1,2] == 0)
eqns.append(x[1,1] == 1)

for i in range(P.n_vertices()):
    eqns.append(sum(x[i,_]^2 for _ in range(d)) == 1)


15


In [202]:
datei = open('./prob.pip', 'w')
datei.write(scip_str(x.values()+[m], eqns, m, bounds))
datei.close()

In [186]:
G = P.graph()
G.is_inscribable()

True

In [230]:
sol = solution('./sol.txt', x.values()+[m])

In [231]:
sol

{x_0_2: 0,
 x_3_0: -0.264143228925214,
 x_5_1: -0.819472900995616,
 x_1_2: 0,
 x_4_0: -0.982923263432983,
 m: 0.435342276698220,
 x_0_1: 0,
 x_2_2: -0.823819822617832,
 x_5_0: 0.154431683100288,
 x_1_1: 1.00000000000000,
 x_3_2: -0.541440770644801,
 x_0_0: 1.00000000000000,
 x_2_1: -0.564505681113061,
 x_4_2: 0.184016226556030,
 x_1_0: -0.0000316228109766838,
 x_3_1: 0.798166858426250,
 x_5_2: -0.551919398242246,
 x_2_0: 0.0515197590590155,
 x_4_1: 2.09073787707040e-9}

In [232]:
Q = Polyhedron([sol[x[i,j]] for j in range(d)] for i in range(P.n_vertices()))

In [227]:
P.plot()+sphere(opacity=.4)

Graphics3d Object

In [233]:
Q.plot() +sphere(opacity=.4)

Graphics3d Object

In [234]:
Q.is_combinatorially_isomorphic(P)

True

In [260]:
def stereo_S_to_R(v):
    return vector([_/(1-v[-1]) for _ in v[:-1]])
def stereo_R_to_S(v):
    dn = 1+sum(_^2 for _ in v)
    return 1/dn*vector([2*_ for _ in v] +[dn - 2])
def rationalize(z):
    return QQ(z).continued_fraction().convergent(3)
def spherify(v):
    return stereo_R_to_S([rationalize(z) for z in stereo_S_to_R(v)])

In [261]:
S  = Polyhedron([spherify(v) for v in Q.vertices()])
S

A 3-dimensional polyhedron in QQ^3 defined as the convex hull of 6 vertices (use the .plot() method to plot)

In [262]:
S.is_inscribed()

True

In [263]:
S.is_combinatorially_isomorphic(P)

True

5/97